In [94]:
from bs4 import BeautifulSoup
import requests
import time, os
import re
import urllib
import json
import pandas as pd
from datetime import datetime
import numpy as np

sites = {'daily': 'https://get61.com/leaderboard', 'lifetime': 'https://get61.com/leaderboard?lifetime=true&sortBy=lifetime_score', 'player_pages': 'https://get61.com/player_profile?player={}'}

userlist = {'Ben Statz' : 'Casetta Bloody Mary', 'Ryan Kelly': 'RKD', 'Gena Rieger-Olson': 'The L in MSLGL',
           'Duncan Sweeney': 'Duncan', 'Jacob Hanke': 'jacobhanke12', 'Edward Mckenna': 'Dr Brad Peck', 'Thomas Gering': 'KingCrab',
           'Andrew Wells': '3SheepsToTheWind', 'Ticho': 'Andrew Ticho', 'Brett Falkowski': 'Falko',
           'Sam Reinertson': 'Riding Lawnmauer', 'Nate Bartz': 'moonblatz', 'Bret Olson': 'Bert Olson',
           'Dan Brunner': 'DPB', 'Srdjan Gajic': 'Provie', 'James Madsen': 'James Madsen', 'Nate Woller': 'Woller',
           'Person1': 'The Merrill Advantage', 'Person2': 'Best Looking Guy in the WVC', 'Denman': 'Denman', 'j':'jbeam1', 'byd':'bigyellowdog'}


In [95]:
def get_leaderboard(site, userlist):
    page = requests.get(site).text
    soup = BeautifulSoup(page, 'html5lib')
    regser = re.compile('var names')
    datajav = soup.find(text=regser)
    
    cats = datajav.split('"')
    
    names = cats[1].split(',')
    score = cats[3].split(',')
    hands = cats[7].split(',')
    pickpct = cats[11].split(',')
    winpct = cats[13].split(',')
    authcodes = cats[5].split(',')
    
    df = pd.DataFrame(list(zip(names, score, hands, pickpct, winpct, authcodes)),
                 columns = ['usernames', 'score', 'hands', 'pickpct', 'winpct', 'authcodes'])
    dflim = df[df.usernames.isin(userlist.values())]
    now = datetime.now()
    dflim['timestamp'] = now.strftime("%m/%d/%Y %H:%M:%S")
    dflim[['score', 'hands', 'pickpct', 'winpct']] = dflim[['score', 'hands', 'pickpct', 'winpct']].apply(pd.to_numeric)

    authdic = dict(zip(dflim['usernames'], dflim['authcodes']))
    return dflim, authdic

In [96]:
scoredf, playercodes = get_leaderboard(sites['lifetime'], userlist)

In [97]:
def crack_count(playercodes):
    summtab = []
    for player in playercodes:
        print(player, playercodes[player])

In [98]:
playercodes

{&#39;Denman&#39;: &#39;auth0|5e8f8fddb3977d0c14d5c2d0&#39;,
 &#39;Dr Brad Peck&#39;: &#39;auth0|5e7ea20b9d8a180c84d8da8b&#39;,
 &#39;Andrew Ticho&#39;: &#39;facebook|10155870343388081&#39;,
 &#39;Bert Olson&#39;: &#39;google-oauth2|115939086903636903039&#39;,
 &#39;jbeam1&#39;: &#39;auth0|5e790278064cbe0c69a88bff&#39;,
 &#39;KingCrab&#39;: &#39;auth0|5e851cf06b269a0bdc0c9bfe&#39;,
 &#39;3SheepsToTheWind&#39;: &#39;google-oauth2|113638829670385164163&#39;,
 &#39;James Madsen&#39;: &#39;google-oauth2|105523472742556596579&#39;,
 &#39;moonblatz&#39;: &#39;auth0|5eaa3ec81cc1ac0c14750c63&#39;,
 &#39;Woller&#39;: &#39;google-oauth2|104264535792617779754&#39;,
 &#39;DPB&#39;: &#39;facebook|10209506157643232&#39;,
 &#39;The Merrill Advantage&#39;: &#39;google-oauth2|118227289723544997064&#39;,
 &#39;Riding Lawnmauer&#39;: &#39;google-oauth2|113453307184472959909&#39;,
 &#39;Best Looking Guy in the WVC&#39;: &#39;google-oauth2|100234991398844308335&#39;,
 &#39;jacobhanke12&#39;: &#39;auth0|5e8

In [99]:
users = []
daily_cracks = []
won_daily_cracks = []
for player in playercodes.values():

    response = requests.get(sites['player_pages'].format(player)).text
    soup = BeautifulSoup(response, 'html5lib')

    regser = re.compile('var numCracks')
    rawsource = soup.find(text=regser)
    rawlisted = rawsource.split(';')
    rawstripped = [row.strip("'\nvar") for row in rawlisted]
    # for row in rawlisted:
    #     rawstripped.append(row.strip("'\nvar"))
    stats = [line.split('=') for line in rawstripped]
    users.append(stats[1][1].replace('"',''))
    daily_cracks.append(stats[44][1].replace('"',''))
    won_daily_cracks.append(stats[67][1].replace('"',''))

crack_df = pd.DataFrame(list(zip(users, daily_cracks, won_daily_cracks)),
                        columns= ['names', 'daily_cracks', 'won_daily_cracks'])
crack_df[['daily_cracks', 'won_daily_cracks']] = crack_df[['daily_cracks', 'won_daily_cracks']].apply(pd.to_numeric)



    
crack_df
    


,names,daily_cracks,won_daily_cracks
0,Denman,3,2
1,Dr Brad Peck,0,0
2,Andrew Ticho,0,0
3,Bert Olson,0,0
4,jbeam1,2,4
5,KingCrab,0,0
6,3SheepsToTheWind,0,0
7,James Madsen,0,0
8,moonblatz,0,0
9,Woller,0,0


In [100]:
crack_df['winpct'] = crack_df['won_daily_cracks'] / crack_df['daily_cracks']

In [101]:
crack_df

,names,daily_cracks,won_daily_cracks,winpct
0,Denman,3,2,0.666667
1,Dr Brad Peck,0,0,NaN
2,Andrew Ticho,0,0,NaN
3,Bert Olson,0,0,NaN
4,jbeam1,2,4,2.000000
5,KingCrab,0,0,NaN
6,3SheepsToTheWind,0,0,NaN
7,James Madsen,0,0,NaN
8,moonblatz,0,0,NaN
9,Woller,0,0,NaN
